In [54]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Flatten
import tensorflow as tf

In [55]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

tf.logging.set_verbosity(tf.logging.INFO)
sess = tf.InteractiveSession()

train = pd.read_csv('Train Data/train.csv')
print('Shape of the train data with all features:', train.shape)
train = train.select_dtypes(exclude=['object'])
print("")
print('Shape of the train data with numerical features:', train.shape)
train.drop('Observation',axis = 1, inplace = True)
train_labels = train['Energy']
print("train labels:",train_labels.shape)
train.fillna(0,inplace=True)
train.drop('Energy',axis = 1, inplace = True)
test = pd.read_csv('Train Data/test.csv')
test = test.select_dtypes(exclude=['object'])
#ID = test.Id
test.fillna(0,inplace=True)
test.drop('Observation',axis = 1, inplace = True)
test_labels = pd.read_csv('Train Data/sample_submission.csv')
test_labels.drop('Observation',axis = 1, inplace = True)
print("test_labels:",test_labels.columns)

print("")
print("List of features contained our dataset train:",list(train.columns))
print("List of features contained our dataset test:",list(test.columns))

Shape of the train data with all features: (15780, 26)

Shape of the train data with numerical features: (15780, 26)
train labels: (15780,)
test_labels: Index(['Energy'], dtype='object')

List of features contained our dataset train: ['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint']
List of features contained our dataset test: ['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint']


In [56]:
train.head(10)

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,22.700000,37.200000,21.000000,38.000000,23.390000,37.290000,22.832857,34.942857,20.500000,42.826667,...,22.700000,45.360000,20.200000,38.663333,12.80,760.050000,62.000000,3.500000,28.0,5.65
1,21.500000,41.045000,20.500000,39.133333,22.926667,39.526667,21.700000,34.126667,18.633333,44.663333,...,22.926667,34.663333,19.730000,37.933333,8.47,764.166667,48.166667,8.000000,26.5,-1.92
2,21.666667,38.000000,22.600000,35.700000,21.890000,36.590000,22.000000,35.530000,19.000000,46.200000,...,21.890000,38.545000,19.790000,39.430000,10.60,757.600000,57.000000,2.000000,27.0,2.40
3,23.290000,38.530000,21.671429,38.000000,24.290000,36.200000,23.100000,35.090000,22.042857,43.625714,...,24.890000,38.600000,22.600000,36.194286,11.60,760.600000,63.000000,1.000000,40.0,4.67
4,24.000000,42.560000,26.830000,34.356000,26.530000,40.333333,24.500000,40.900000,21.500000,49.090000,...,23.390000,44.230000,22.600000,44.090000,17.70,756.983333,63.000000,1.000000,21.5,10.40
5,18.390000,43.363333,17.700000,42.433333,19.290000,42.500000,18.600000,41.663333,17.200000,55.360000,...,18.500000,47.600000,16.700000,46.000000,1.70,749.400000,97.000000,3.833333,32.5,1.25
6,24.500000,36.060000,24.434000,34.634000,24.600000,33.590000,24.890000,34.700000,24.465714,80.110000,...,25.356667,37.626667,23.405714,34.880000,9.43,756.300000,61.666667,5.000000,40.0,2.40
7,23.000000,34.863333,21.000000,35.223333,23.000000,36.090000,22.633333,33.933333,21.390000,53.260000,...,24.100000,39.363333,20.318571,37.200000,8.00,765.950000,68.166667,1.000000,40.0,2.33
8,23.100000,35.000000,23.390000,31.856667,23.000000,35.290000,23.000000,33.200000,20.856667,38.363333,...,21.666667,37.863333,20.290000,36.700000,18.60,760.600000,38.000000,4.500000,40.0,4.05
9,21.000000,37.200000,18.500000,40.656667,20.390000,37.260000,21.600000,34.463333,20.093333,78.800000,...,21.000000,38.123333,17.290000,35.590000,4.67,748.833333,72.666667,3.666667,40.0,0.20


In [57]:
import warnings
warnings.filterwarnings('ignore')

col_train = list(train.columns)
col_train_bis = list(train.columns)

#col_train_bis.remove('SalePrice')

mat_train = np.matrix(train)
mat_test  = np.matrix(test)
mat_y_test = np.array(test_labels).reshape((3945,1))
mat_y_train = np.array(train_labels).reshape((15780,1))

prepro_y_train = MinMaxScaler()
prepro_y_train.fit(mat_y_train)

prepro_y_test = MinMaxScaler()
prepro_y_test.fit(mat_y_test)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_test)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
test  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis)
y_train = pd.DataFrame(prepro_y_train.transform(mat_y_train),columns = ['Energy'])
y_test = pd.DataFrame(prepro_y_test.transform(mat_y_test),columns = ['Energy'])

train.head()
#test.head()
#y_train.head()
#y_test.head()

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,0.624076,0.280066,0.356191,0.493111,0.514290,0.398349,0.696654,0.310835,0.496399,0.195645,...,0.583171,0.540096,0.552549,0.393074,0.572347,0.715116,0.500000,0.250000,0.415385,0.554299
1,0.497360,0.385882,0.319845,0.524979,0.475795,0.502882,0.594595,0.275999,0.317171,0.223261,...,0.604028,0.173521,0.503642,0.362859,0.433119,0.810853,0.317982,0.571429,0.392308,0.211765
2,0.514960,0.302082,0.472498,0.428438,0.389664,0.365633,0.621622,0.335894,0.352376,0.246366,...,0.508639,0.306546,0.509886,0.424807,0.501608,0.658140,0.434211,0.142857,0.400000,0.407240
3,0.686378,0.316668,0.404998,0.493111,0.589066,0.347406,0.720721,0.317115,0.644537,0.207659,...,0.784685,0.308430,0.802289,0.290878,0.533762,0.727907,0.513158,0.071429,0.600000,0.509955
4,0.761352,0.427575,0.779985,0.390646,0.775174,0.540583,0.846847,0.565087,0.592415,0.289821,...,0.646662,0.501371,0.802289,0.617688,0.729904,0.643798,0.513158,0.071429,0.315385,0.769231


In [92]:
print(train.shape)
print(test.shape)
print(y_train.shape)
print(y_test.shape)
model = Sequential()
model.add(Dense(32,input_shape = train.shape,activation='relu'))
model.add(Flatten())
#model.add(Activation('relu'))
model.add(Dense(1,activation='tanh'))
#model.add(Activation('tanh'))

(15780, 24)
(3945, 24)
(15780, 1)
(3945, 1)


In [93]:
model.compile(optimizer='rmsprop',
              loss='mse')

In [94]:
model.fit(train, y_train, epochs=10, batch_size=32)

ValueError: Error when checking input: expected dense_35_input to have 3 dimensions, but got array with shape (15780, 24)